데이터 : data_atype.zip (수업노트에서 다운로드)

## 머신러닝
- 문제정의, 라이브러리/데이터 불러오기
- 탐색적 데이터 분석 (EDA)
- 데이터 전처리
- 피처엔지니어링
- (Train/Validation 나누기)
- 모델 선택/훈련/평가/최적화
- 예측
- (csv 생성)

## 1. 베이스라인
- 문제정의, 라이브러리 및 데이터 불러오기
- 데이터 전처리 (단순 일괄 처리)
- 모델 선택, 훈련
- 평가

## 2. 베이스라인
- 훈련/검증용 데이터 분리
- 모델 선택, 훈련
    - 의사결정나무
    - 랜덤포레스트
    - XGBoost
- 평가

## 문제1
- "<= 50K -> 0"
- "> 50K -> 1"
- 평가: 정확도

In [62]:
# 라이브러리 및 데이터 불러오기
# import pandas as pd
# X_train = pd.read_csv("X_train.csv")
# X_test = pd.read_csv("X_test.csv")
# y_train = pd.read_csv("y_train.csv")

#라이브러리
import pandas as pd
import numpy as np

# 데이터(훈련x,훈련y,테스트x)
X_train = pd.read_csv("X_train.csv") # 모델이 학습할 때 사용  ex) 이미지
y_train = pd.read_csv("y_train.csv") # 모델이 예측해야할 데이터  ex) 고양이, 개
 # x_train과 y_train을 사용해 학습 -> 입력 데이터와 정답 데이터 간관계 학습
X_test = pd.read_csv("X_test.csv") # 모델 성능 평가에 사용되는 데이터 -> 훈련과정에 사용X
 # 추후 y_test데이터를 사용하여 해당 모델 성능 평가

In [63]:
# 데이터 크기
# X_train.shape, X_test.shape, y_train.shape

# 데이터 크기 분석
X_train.shape, y_train.shape, X_test.shape
# y_train 칼럼은 2개
# 나머지 두 개는 칼럼 15개로 일치

((29304, 15), (29304, 2), (3257, 15))

In [64]:
# 데이터 샘플
# X_train.head()
#y_train.head() - id, income

In [65]:
# 타겟 수 확인
# y_train['income'].value_counts()

# 처음 타겟이 모르겠다면 ? -> y_train 사용

# 타겟 수 확인
y_train['income'].value_counts()
# 소득 50K이하가 많음

income
<=50K    22263
>50K      7041
Name: count, dtype: int64

In [66]:
# type확인
# X_train.info()

# 피처 타입 확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 3.4+ MB


In [67]:
# 수치형 데이터
# cols =[
#        'age',
#        'fnlwgt',
#        'education.num',
#        'capital.gain',
#        'capital.loss',
#        'hours.per.week'
# ]

# 수치형 데이터 뽑기
cols = list(X_train.select_dtypes(exclude="object").columns)
cols = cols[1:]
cols

['age',
 'fnlwgt',
 'education.num',
 'capital.gain',
 'capital.loss',
 'hours.per.week']

In [68]:
# 수치형 데이처 통계
# X_train[cols].describe()

# 수치형 데이터 통계 보기 -> 결측치 / 이상치 파악
X_train[cols].describe()
# age에 음수값이 있다..

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [69]:
# 결측값
# X_train[cols].isnull().sum()

# 결측값 파악
X_train[cols].isnull().sum()
# age와 hours.per.week에 결측값 존재

age               12
fnlwgt             0
education.num      0
capital.gain       0
capital.loss       0
hours.per.week    13
dtype: int64

In [70]:
# 간단한 결측치 처리 -> 그냥 0으로 채우기
# X_train = X_train.fillna(0)
# X_test = X_test.fillna(0)

X_train[cols] = X_train[cols].fillna(0)
X_test[cols] = X_test[cols].fillna(0)

In [71]:
# 결측치 확인
X_train[cols].isnull().sum()

age               0
fnlwgt            0
education.num     0
capital.gain      0
capital.loss      0
hours.per.week    0
dtype: int64

In [72]:
# 베이스 라인에서는 그외 전처리 및 피처 엔지니어링 생략

In [73]:
# target값 변경
# <=50K -> 0
# >50K -> 1
# y = (X_train['income'] != '<=50K').astype(int)

# y_train['income'].value_counts()
y = (y_train['income'] != '<=50K').astype(int)
y

0        1
1        0
2        0
3        1
4        0
        ..
29299    0
29300    0
29301    0
29302    0
29303    0
Name: income, Length: 29304, dtype: int64

In [74]:
y = (y_train['income'] == '>50K').astype(int)
y[:3]

0    1
1    0
2    0
Name: income, dtype: int64

In [75]:
X_train[cols]

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0
2,48.0,125933,10,0,1669,38.0
3,58.0,100313,10,0,1902,40.0
4,41.0,195661,10,0,0,54.0
...,...,...,...,...,...,...
29299,28.0,47168,6,0,0,40.0
29300,44.0,231793,16,0,0,38.0
29301,41.0,201435,9,0,0,40.0
29302,43.0,137722,9,0,0,40.0


## 머신러닝 모델

In [76]:
#분류모델
#목표 : 특정 범주나 클래스 예측
#target 변수 유형 : 범주형 데이터
#예시 : 이진분류(이메일 스팸 유무 / 구매유무), 다중 클래스 분류(고양이, 개, 새 / 질병 종류)
#회귀모델
#목표 : 연속적인 수치 예측
#target 변수 유형 : 연속형 데이터
#예시 ; 가격 예측, 온도 예측, 매출액 에측,,,


# 랜덤포레스트
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# rf.fit(X_train[cols], y)
# pred = rf.predict(X_test[cols])

# 랜덤포레스트 분류 모델 만들기
from sklearn.ensemble import RandomForestClassifier
# 모델 가져오기
rf = RandomForestClassifier()
# 모델 학습
rf.fit(X_train[cols], y)
# 모델 예측
pred = rf.predict(X_test[cols])
# len(X_test)


In [77]:
# len(X_test)

In [78]:
# 예측 및 csv 파일 생성
# pred[:10]
# submit = pd.DataFrame(
#             {
#                 'id':X_test['id'],
#                 'income':pred
#             }
#         )
# submit.to_csv("11111.csv", index=False)

# 예측된 결과 확인
# pred[:10]

#csv파일 생성
submit = pd.DataFrame(
    {
        "id" : X_test['id'], #X_test임 주의 -> pred는 테스트 데이터로 예측해서 나온 target값이니까
        "income" : pred
    }
)

submit.to_csv("0000.csv", index=False)

In [79]:
y_train.head()

,id,income
0,3331,>50K
1,19749,<=50K
2,1157,<=50K
3,693,>50K
4,12522,<=50K


In [80]:
# 평가 (수험자는 알 수 없는 부분임) accuracy
from sklearn.metrics import accuracy_score
y_test = pd.read_csv("y_test.csv")
ans = (y_test['income'] == '>50K').astype(int)
accuracy_score(ans, pred)
# 수치형 데이터만 썼는데 괜찮은 정확도가 나왔군!

0.8114829597789377

In [81]:
y_test

,id,income
0,11574,<=50K
1,15847,<=50K
2,17655,<=50K
3,19790,>50K
4,31812,<=50K
...,...,...
3252,20589,<=50K
3253,5668,<=50K
3254,27652,<=50K
3255,14735,<=50K


## 문제2
- "<= 50K -> 0"
- "> 50K -> 1"
- 평가: roc_auc 예측 해야할 값은 : 확률

## 검증용 데이터 분리

In [97]:
# 학습용 데이터와 검증용 데이터로 구분
# from sklearn.model_selection import train_test_split
# y = (y_train['income'] == '>50K').astype(int)
# X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.1, random_state=2022)

# 자체 평가를 하기 위해 -> 훈련용 데이터에서 검증영 데이터 구분하기
from sklearn.model_selection import train_test_split
y = (y_train['income'] == '>50K').astype(int)
# 데이터 분리
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.1, random_state=2022)


# 평가방식 roc_auc
# 이진 분류 모델 성능 평가시 사용되는 지표 -> 값이 클수록 좋은 성능 의미
# 확률 값을 예측할 때는 -> predict_proba사용

In [98]:
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((26373, 15), (2931, 15), (26373,), (2931,))

In [120]:
# 분류 모델 : 의사결정나무 DecisionTreeClassifier
# from sklearn.tree import DecisionTreeClassifier
# dt = DecisionTreeClassifier()
# dt.fit(X_tr[cols], y_tr)
# pred=dt.predict_proba(X_val[cols])
# roc_auc_score(y_val,pred[:,1])

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_tr[cols], y_tr)
pred = dt.predict_proba(X_val[cols])
pred

#문제에서 주어진다 -> 0일확률을 구하는건지? 1일 확률 구하는 건지?
# 0일 확률 구한다 : pred[:,0]
# 1일 확률 구한다 : pred[:,1]

from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:, 1])

0.6991435256054095

In [121]:
# 랜덤포레스트
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# rf.fit(X_tr[cols], y_tr)
# pred=rf.predict_proba(X_val[cols])
# roc_auc_score(y_val,pred[:,1])

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_tr[cols], y_tr)
pred = rf.predict_proba(X_val[cols])
pred

from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:,1])

0.8510864676289719

In [123]:
# XGBoost
# from xgboost import XGBClassifier
# xgb = XGBClassifier()
# xgb.fit(X_tr[cols], y_tr)
# pred=xgb.predict_proba(X_val[cols])
# pred[:,1]

from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_tr[cols], y_tr)
pred = xgb.predict_proba(X_val[cols])
print(pred) # 0과1로 예측되는데
print(pred[:, 1])

from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:, 1])

[[0.9618588  0.03814121]
 [0.9944762  0.00552378]
 [0.52776355 0.47223645]
 ...
 [0.8434911  0.15650894]
 [0.9610678  0.03893219]
 [0.00963706 0.99036294]]
[0.03814121 0.00552378 0.47223645 ... 0.15650894 0.03893219 0.99036294]


0.8859038904196574

In [130]:
# 평가 데이터로 예측 및 csv파일 생성
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_val,pred[:,1])

# pred=xgb.predict_proba(X_test[cols])
# submit = pd.DataFrame(
#             {
#                 'id':X_test['id'],
#                 'income':pred[:,1]
#             }
#         )
# submit.to_csv("22222.csv", index=False)

#XGBoost가 제일 높으니까 XGBoost모델 사용
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_tr[cols], y_tr)
pred = xgb.predict_proba(X_test[cols])
# print(pred)

# print(y_train)

submit = pd.DataFrame({
    "id" : X_test['id'],
    "income" : pred[:, 1]
})

submit.to_csv("2222.csv", index=False)

[[0.66851985 0.33148018]
 [0.8210815  0.1789185 ]
 [0.7379545  0.26204547]
 ...
 [0.9696596  0.03034037]
 [0.9075354  0.09246464]
 [0.9747738  0.02522616]]


# 평가
- 수험자는 알 수 없는 영역임

In [ ]:
from sklearn.metrics import roc_auc_score
y_test = pd.read_csv("y_test.csv")
ans = (y_test['income'] != '<=50K').astype(int)
roc_auc_score(ans, pred[:,1])

0.8776427045177047